---

# **Notebook 14 : Nettoyage et Sélection des Catégories et Groupes PNNS pour les Produits**

---

### **1. Importation des Bibliothèques**

Nous commençons par importer les bibliothèques nécessaires pour le traitement des fichiers et la manipulation des données.

In [1]:
import os
import pandas as pd
import re

---

### **2. Définition des Répertoires d'Entrée et de Sortie**

Nous définissons les chemins pour le répertoire contenant les fichiers CSV à traiter et celui où nous sauvegarderons les fichiers nettoyés.

In [ ]:
# Répertoires d'entrée et de sortie
input_directory = 'note-books/phase_1_data/08_data_countries'
output_directory = 'note-books/phase_1_data/14_categorized_data'
os.makedirs(output_directory, exist_ok=True)

---

### **3. Fonction de Nettoyage des Colonnes `categories`, `pnns_groups_1`, et `pnns_groups_2`**

Cette fonction applique les règles de nettoyage suivantes :

1. **Suppression du préfixe `en:`** dans la colonne `categories`.
2. **Conservation des accents et caractères spéciaux courants** (apostrophes, par exemple) dans les noms de catégories.
3. **Sélection de la première catégorie** dans `categories` si plusieurs sont présentes (sauf si la première catégorie est `"unknown"`).
4. **Remplacement des valeurs non conformes** ou celles contenant des caractères invalides par `"unknown"` dans toutes les colonnes.

In [ ]:
def nettoyer_valeur(valeur):
    if pd.isna(valeur) or valeur.strip() == "":
        return "unknown"  # Remplace les valeurs manquantes par "unknown"
    
    # Supprimer le préfixe "en:" uniquement pour les catégories
    valeur = re.sub(r'^en:', '', valeur)
    
    # Conserver les caractères latins avec accents et apostrophes
    valeur = re.sub(r"[^a-zA-Zéèêëàâäîïôöùûüç'\s,]", '', valeur).strip()
    
    # Si plusieurs valeurs sont séparées par des virgules, garder la première non "unknown"
    valeurs_list = [v.strip() for v in valeur.split(',') if v.strip()]
    first_valid_value = next((v for v in valeurs_list if v.lower() != "unknown"), "unknown")
    return first_valid_value

---

### **4. Application de la Fonction de Nettoyage et Création des Nouveaux Fichiers**

Nous parcourons chaque fichier CSV dans le répertoire d'entrée, appliquons le nettoyage aux colonnes `categories`, `pnns_groups_1`, et `pnns_groups_2`, et sauvegardons les données nettoyées dans le répertoire de sortie.


In [ ]:
# Parcourir les fichiers et appliquer le nettoyage sur les colonnes spécifiées
for file_name in os.listdir(input_directory):
    file_path = os.path.join(input_directory, file_name)
    output_file_path = os.path.join(output_directory, file_name)
    
    # Charger le fichier avec gestion de l'encodage
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        print(f"Problème d'encodage pour {file_name}, réessayé avec 'ISO-8859-1'.")
        df = pd.read_csv(file_path, encoding='ISO-8859-1', errors='replace')
    
    # Appliquer la fonction de nettoyage aux colonnes 'categories', 'pnns_groups_1', et 'pnns_groups_2'
    for col in ['categories', 'pnns_groups_1', 'pnns_groups_2']:
        if col in df.columns:
            df[col] = df[col].apply(nettoyer_valeur)
    
    # Sauvegarder le fichier nettoyé
    df.to_csv(output_file_path, index=False)
    print(f"Fichier nettoyé et sauvegardé : {output_file_path}")

---

### **5. Explications**

- **Suppression des préfixes `en:`** : Nous appliquons cette suppression uniquement à la colonne `categories`.
- **Sélection de la première valeur valide** : Seule la première valeur est conservée dans chaque colonne si plusieurs sont présentes, sauf si elle est `unknown`.
- **Caractères spéciaux autorisés** : Les accents, apostrophes et espaces sont conservés pour plus de précision dans les noms.
- **Remplacement des valeurs non conformes** : Les cellules contenant des caractères non valides ou vides sont remplacées par `"unknown"`.

--- 